Create a few simple tests to test the speed of using openAI or Google Translate or HuggingFace (deepsink) API for translation tasks from nepali to english

In [ ]:
#create a simple dataset of 1000 sentences in nepali
import pandas as pd
import numpy as np
import random
import time
import requests
import json
import os
from icecream import ic
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HF_API_KEY = os.getenv('HUGGINGFACE_API_KEY')
#load the dataset
df_nepali = pd.read_csv('/home/ubuntu/nepal_chatbot/translation/sentiment_analysis_nepali_final.csv').drop(columns=['Unnamed: 0'])




In [ ]:
df_nepali.head()

In [ ]:
# create a function to call google translate api
def translate_nepali_to_english(text):
    url = 'https://translation.googleapis.com/language/translate/v2'
    t0 = time.time()
    params = {
        'q': text,
        'source': 'ne',
        'target': 'en',
        'key': GOOGLE_API_KEY
    }
    translation_time = time.time() - t0
    response = requests.get(url, params=params)
    # print(response.json())
    return response.json()['data']['translations'][0]['translatedText'], translation_time

#test the function
for i in range(10):
    print(i, translate_nepali_to_english(df_nepali['Sentences'][i])[1], translate_nepali_to_english(df_nepali['Sentences'][i])[0])


In [ ]:
import os
import openai
from typing import Optional

def translate_to_english_openai(nepali_text: str) -> Optional[str]:
    """
    Translate Nepali text to English using OpenAI's API.
    
    Args:
        nepali_text (str): The Nepali text to translate
        
    Returns:
        Optional[str]: The translated English text, or None if translation fails
    """
    try:
        # Initialize OpenAI client
        client = openai.OpenAI(
            api_key=os.getenv('OPENAI_API_KEY')
        )
        
        # Create the prompt for translation
        prompt = f"""Translate the following Nepali text to English. 
        Only provide the translation without any additional text or explanations.
        
        Nepali text: {nepali_text}
        
        English translation:"""
        
        # Call OpenAI API
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a professional Nepali to English translator. Provide only the translation without any additional text. Provide the translation in a natural way."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150
        )
        
        # Extract and return the translation
        translation = response.choices[0].message.content.strip()
        return translation
        
    except Exception as e:
        print(f"Translation error: {str(e)}")
        return None

def translate_nepali_to_english_openai(text):
    t0 = time.time()
    translation = translate_to_english_openai(text)
    translation_time = time.time() - t0
    return translation, translation_time

for i in range(10):
    print(i, translate_nepali_to_english_openai(df_nepali['Sentences'][i])[1], translate_nepali_to_english_openai(df_nepali['Sentences'][i])[0])



In [ ]:
import requests
import json
from huggingface_hub import hf_hub_download
from datetime import datetime, timedelta

url = "http://<your-server-url>:8000/v1/chat/completions"
headers = {"Content-Type": "application/json", "Authorization": "Bearer token"}

model = "mistralai/Mistral-Small-3.1-24B-Instruct-2503"


def load_system_prompt(repo_id: str, filename: str) -> str:
    file_path = hf_hub_download(repo_id=repo_id, filename=filename)
    with open(file_path, "r") as file:
        system_prompt = file.read()
    today = datetime.today().strftime("%Y-%m-%d")
    yesterday = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
    model_name = repo_id.split("/")[-1]
    return system_prompt.format(name=model_name, today=today, yesterday=yesterday)


SYSTEM_PROMPT = load_system_prompt(model, "SYSTEM_PROMPT.txt")

image_url = "https://huggingface.co/datasets/patrickvonplaten/random_img/resolve/main/europe.png"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Which of the depicted countries has the best food? Which the second and third and fourth? Name the country, its color on the map and one its city that is visible on the map, but is not the capital. Make absolutely sure to only name a city that can be seen on the map.",
            },
            {"type": "image_url", "image_url": {"url": image_url}},
        ],
    },
]

data = {"model": model, "messages": messages, "temperature": 0.15}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json()["choices"][0]["message"]["content"])
# Determining the "best" food is highly subjective and depends on personal preferences. However, based on general popularity and recognition, here are some countries known for their cuisine:

# 1. **Italy** - Color: Light Green - City: Milan
#    - Italian cuisine is renowned worldwide for its pasta, pizza, and various regional specialties.

# 2. **France** - Color: Brown - City: Lyon
#    - French cuisine is celebrated for its sophistication, including dishes like coq au vin, bouillabaisse, and pastries like croissants and éclairs.

# 3. **Spain** - Color: Yellow - City: Bilbao
#    - Spanish cuisine offers a variety of flavors, from paella and tapas to jamón ibérico and churros.

# 4. **Greece** - Not visible on the map
#    - Greek cuisine is known for dishes like moussaka, souvlaki, and baklava. Unfortunately, Greece is not visible on the provided map, so I cannot name a city.

# Since Greece is not visible on the map, I'll replace it with another country known for its good food:

# 4. **Turkey** - Color: Light Green (east part of the map) - City: Istanbul
#    - Turkish cuisine is diverse and includes dishes like kebabs, meze, and baklava.


In [ ]:


from server_utils import get_server_url, save_server_urls, load_server_urls
# Get all available URLs
urls = get_server_url(port=5005, protocol="http")
print(urls)
# Output example:
# {
#     "local": "http://192.168.1.100:5005",
#     "localhost": "http://localhost:5005",
#     "127.0.0.1": "http://127.0.0.1:5005",
#     "public": "http://13.212.33.134:5005"
# }

# Save URLs to file
save_server_urls(urls)

# Load URLs from file
loaded_urls = load_server_urls()

In [3]:
from utterance_mapping import get_utterance
from icecream import ic

language_code = "en"
ic(get_utterance("grievance_form", 
                    "create_confirmation_message", 
                    "grievance_summary", 
                    language_code))
ic(get_utterance("grievance_form", 
                    "create_confirmation_message", 
                    "grievance_id", 
                    language_code))
message_list = []
for i in ['grievance_id',
          'grievance_summary',
          'grievance_category',
          'grievance_details',
          'grievance_email',
          'grievance_phone',
          'grievance_outro']:
    print(get_utterance("grievance_form", 
                                 'create_confirmation_message', 
                                 i, 
                                 language_code))
    message_list.append(get_utterance("grievance_form", 
                                 'create_confirmation_message', 
                                 i, 
                                 language_code))
    
print(message_list)

message = [get_utterance("grievance_form", 
                                 'create_confirmation_message', 
                                 i, 
                                 language_code) for i in ['grievance_id',
                                                         'grievance_summary',
                                                         'grievance_category',
                                                         'grievance_details',
                                                         'grievance_email',
                                                         'grievance_phone',
                                                         'grievance_outro']]

print(message)

ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_summary'
    language: 'en'
ic| get_utterance("grievance_form", 
                     "create_confirmation_message", 
                     "grievance_summary", 
                     language_code): '**Summary: {grievance_summary}**'
ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_id'
    language: 'en'
ic| get_utterance("grievance_form", 
                     "create_confirmation_message", 
                     "grievance_id", 
                     language_code): 'Your grievance has been filed successfully.
                                      **Grievance ID: {grievance_id} **'
ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_id'
    language: 'en'
ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_id'
    la

Your grievance has been filed successfully.
**Grievance ID: {grievance_id} **
**Summary: {grievance_summary}**
**Category: {grievance_category}**
**Details: {grievance_details}**


    utter_index: 'grievance_details'
    language: 'en'
ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_email'
    language: 'en'
ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_email'
    language: 'en'
ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_phone'
    language: 'en'
ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_phone'
    language: 'en'
ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_outro'
    language: 'en'
ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_outro'
    language: 'en'
ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_id'
    language: 'en'
ic| form_name: '


A confirmation email will be sent to {grievance_email}
**Phone: {grievance_phone}**
Our team will review it shortly and contact you if more information is needed.
['Your grievance has been filed successfully.\n**Grievance ID: {grievance_id} **', '**Summary: {grievance_summary}**', '**Category: {grievance_category}**', '**Details: {grievance_details}**', '\nA confirmation email will be sent to {grievance_email}', '**Phone: {grievance_phone}**', 'Our team will review it shortly and contact you if more information is needed.']


'
    language: 'en'
ic| form_name: 'grievance_form'
    action_name: 'create_confirmation_message'
    utter_index: 'grievance_outro'
    language: 'en'


['Your grievance has been filed successfully.\n**Grievance ID: {grievance_id} **', '**Summary: {grievance_summary}**', '**Category: {grievance_category}**', '**Details: {grievance_details}**', '\nA confirmation email will be sent to {grievance_email}', '**Phone: {grievance_phone}**', 'Our team will review it shortly and contact you if more information is needed.']
